In [1]:
import subprocess
import json
import pandas as pd

def describe_instances_to_dataframe():
    # Run AWS CLI command to describe instances
    command = "aws ec2 describe-instances --region=us-west-2"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print("Error:", result.stderr)
        return None

    # Parse JSON output
    try:
        instances_data = json.loads(result.stdout)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return None

    # Extract relevant information for DataFrame
    instances_list = []
    for reservation in instances_data.get('Reservations', []):
        for instance in reservation.get('Instances', []):
            instance_info = {
                'InstanceId': instance.get('InstanceId', ''),
                'InstanceType': instance.get('InstanceType', ''),
                'State': instance.get('State', {}).get('Name', ''),
                'PublicIpAddress': instance.get('PublicIpAddress', ''),
                'PrivateIpAddress': instance.get('PrivateIpAddress', ''),
                # Add more fields as needed
            }
            instances_list.append(instance_info)

    # Create Pandas DataFrame
    df = pd.DataFrame(instances_list)
    return df

# Run the function
df = describe_instances_to_dataframe()

# Display the DataFrame
if df is not None:
    print(df)


             InstanceId  InstanceType    State PublicIpAddress  \
0   i-09f29582378fa661e    t3a.medium  stopped                   
1   i-0212d6ca67b8e1493   t3a.2xlarge  stopped                   
2   i-0fddf52d4478ddf06      t3.micro  stopped                   
3   i-052595b138b9def8e      t3.large  stopped                   
4   i-064ffc3c21b6fb48b    t3a.xlarge  stopped                   
5   i-0e500fefdaa2603e1     m6i.large  stopped                   
6   i-0f7081ffe14e32346     m6i.large  stopped                   
7   i-0e58ee63cbd0d8bf0      t3.large  stopped                   
8   i-01dccf67b0459d8e7      t3.large  running                   
9   i-06e37d49029cce732  m6a.12xlarge  stopped                   
10  i-08ae4097b2069ce5b   m6a.2xlarge  stopped                   
11  i-0be567382c55be0c8   t3a.2xlarge  stopped                   
12  i-0fae66f82c4bbd5a0    m5d.xlarge  running                   
13  i-0138c8d8be3c548bf      t3.large  running                   
14  i-0e5e

In [2]:
import boto3

def get_instance_name(instance_id):
    # Specify the region
    region = 'us-west-2'  # Replace with your AWS region, e.g., 'us-east-1'

    # Create EC2 client
    ec2 = boto3.client('ec2', region_name=region)

    try:
        # Describe the instance to get its tags
        response = ec2.describe_instances(InstanceIds=[instance_id])

        # Extract the "Name" tag value
        for reservation in response['Reservations']:
            for instance in reservation['Instances']:
                for tag in instance.get('Tags', []):
                    if tag['Key'] == 'Name':
                        return tag['Value']

        # If the instance has no "Name" tag
        return f"No 'Name' tag found for instance {instance_id}"

    except Exception as e:
        return f"Error: {str(e)}"

# Replace 'your-instance-id' with the actual instance ID
# instance_id = 'i-05eca5a0ba83f964b'
# instance_name = get_instance_name(instance_id)

# print(f"The 'Name' tag for instance {instance_id} is: {instance_name}")


In [3]:
for i,r in df.iterrows():
    print(r['InstanceId'])
    Name = get_instance_name(r['InstanceId'])
    print(Name)

i-09f29582378fa661e
nlcd-pc-adminbox
i-0212d6ca67b8e1493
nlcd-kpostma-system-dev
i-0fddf52d4478ddf06
nlcd-bschulz-system-dev
i-052595b138b9def8e
nlcd-pcluster-access-host
i-064ffc3c21b6fb48b
nlcd-data-xfer-dev
i-0e500fefdaa2603e1
nlcd-dzelenak-system-dev
i-0f7081ffe14e32346
nlcd-caustin-system-dev
i-0e58ee63cbd0d8bf0
HeadNode-crobinson-pcluster-v2
i-01dccf67b0459d8e7
HeadNode-pdanielson-pcluster
i-06e37d49029cce732
nlcd-docker-ship-dev
i-08ae4097b2069ce5b
nlcd-tbutzer-system-dev
i-0be567382c55be0c8
INL-pangeo
i-0fae66f82c4bbd5a0
dev-nlcd-eks-ec2-cluster-lsis-system-nodes-persistent-a--eks_asg
i-0138c8d8be3c548bf
HeadNode-butzer-pcluster-v5
i-0e5e4c42949e69586
Compute


In [4]:
df['InstanceName'] = df['InstanceId'].apply(get_instance_name)

In [5]:
df

,InstanceId,InstanceType,State,PublicIpAddress,PrivateIpAddress,InstanceName
0,i-09f29582378fa661e,t3a.medium,stopped,,10.12.71.39,nlcd-pc-adminbox
1,i-0212d6ca67b8e1493,t3a.2xlarge,stopped,,10.12.71.7,nlcd-kpostma-system-dev
2,i-0fddf52d4478ddf06,t3.micro,stopped,,10.12.70.5,nlcd-bschulz-system-dev
3,i-052595b138b9def8e,t3.large,stopped,,10.12.70.32,nlcd-pcluster-access-host
4,i-064ffc3c21b6fb48b,t3a.xlarge,stopped,,10.12.71.250,nlcd-data-xfer-dev
5,i-0e500fefdaa2603e1,m6i.large,stopped,,10.12.71.183,nlcd-dzelenak-system-dev
6,i-0f7081ffe14e32346,m6i.large,stopped,,10.12.71.249,nlcd-caustin-system-dev
7,i-0e58ee63cbd0d8bf0,t3.large,stopped,,10.12.71.200,HeadNode-crobinson-pcluster-v2
8,i-01dccf67b0459d8e7,t3.large,running,,10.12.70.56,HeadNode-pdanielson-pcluster
9,i-06e37d49029cce732,m6a.12xlarge,stopped,,10.12.70.96,nlcd-docker-ship-dev


In [6]:
selected_rows = df[df['InstanceName'].isin(['HeadNode', 'Compute'])]

In [7]:
selected_rows

,InstanceId,InstanceType,State,PublicIpAddress,PrivateIpAddress,InstanceName
14,i-0e5e4c42949e69586,m6i.2xlarge,running,,10.12.65.76,Compute


In [8]:
import socket

In [9]:
hostname = socket.gethostname()

In [10]:
hostname

'ip-10-12-65-76'

In [11]:
ip_address = socket.gethostbyname(hostname)

In [12]:
ip_address

'10.12.65.76'

In [13]:
selected_rows = df[df['PrivateIpAddress'].isin([ip_address, ])]

In [14]:
selected_rows

,InstanceId,InstanceType,State,PublicIpAddress,PrivateIpAddress,InstanceName
14,i-0e5e4c42949e69586,m6i.2xlarge,running,,10.12.65.76,Compute


In [27]:
my_new_tag = 'Compute-pdanielson-composite'
awsid = selected_rows['InstanceId'].values[0]

In [28]:
awsid

'i-0e5e4c42949e69586'

In [32]:
command = f'aws ec2 create-tags --resources {awsid} --tag \"Key=Name,Value={my_new_tag}\" --region=us-west-2'
result = subprocess.run(command, shell=True, capture_output=True, text=True)

# Check for errors
if result.returncode != 0:
    print("Error:", result.stderr)

In [33]:
df = describe_instances_to_dataframe()


In [34]:
df

,InstanceId,InstanceType,State,PublicIpAddress,PrivateIpAddress
0,i-09f29582378fa661e,t3a.medium,stopped,,10.12.71.39
1,i-0212d6ca67b8e1493,t3a.2xlarge,stopped,,10.12.71.7
2,i-0fddf52d4478ddf06,t3.micro,stopped,,10.12.70.5
3,i-052595b138b9def8e,t3.large,stopped,,10.12.70.32
4,i-064ffc3c21b6fb48b,t3a.xlarge,stopped,,10.12.71.250
5,i-0e500fefdaa2603e1,m6i.large,stopped,,10.12.71.183
6,i-0f7081ffe14e32346,m6i.large,stopped,,10.12.71.249
7,i-0e58ee63cbd0d8bf0,t3.large,stopped,,10.12.71.200
8,i-01dccf67b0459d8e7,t3.large,running,,10.12.70.56
9,i-06e37d49029cce732,m6a.12xlarge,stopped,,10.12.70.96


In [35]:
df['InstanceName'] = df['InstanceId'].apply(get_instance_name)

In [36]:
df

,InstanceId,InstanceType,State,PublicIpAddress,PrivateIpAddress,InstanceName
0,i-09f29582378fa661e,t3a.medium,stopped,,10.12.71.39,nlcd-pc-adminbox
1,i-0212d6ca67b8e1493,t3a.2xlarge,stopped,,10.12.71.7,nlcd-kpostma-system-dev
2,i-0fddf52d4478ddf06,t3.micro,stopped,,10.12.70.5,nlcd-bschulz-system-dev
3,i-052595b138b9def8e,t3.large,stopped,,10.12.70.32,nlcd-pcluster-access-host
4,i-064ffc3c21b6fb48b,t3a.xlarge,stopped,,10.12.71.250,nlcd-data-xfer-dev
5,i-0e500fefdaa2603e1,m6i.large,stopped,,10.12.71.183,nlcd-dzelenak-system-dev
6,i-0f7081ffe14e32346,m6i.large,stopped,,10.12.71.249,nlcd-caustin-system-dev
7,i-0e58ee63cbd0d8bf0,t3.large,stopped,,10.12.71.200,HeadNode-crobinson-pcluster-v2
8,i-01dccf67b0459d8e7,t3.large,running,,10.12.70.56,HeadNode-pdanielson-pcluster
9,i-06e37d49029cce732,m6a.12xlarge,stopped,,10.12.70.96,nlcd-docker-ship-dev
